# imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import copy
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

# Hyperparameters
random_seed = 123

torch.manual_seed(random_seed)

# Exercise 1: Implementing the LoRALayer

In [2]:
class LoRALayer(nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        # Инициализируем матрицу A с нормальным распределением, масштабированным 1/sqrt(rank)
        # Это помогает поддерживать норму активаций
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.A = nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        # Инициализируем матрицу B нулями
        # Это гарантирует, что в начале адаптация LoRA равна нулю,
        # и модель начинает обучение с исходных весов.
        self.B = nn.Parameter(torch.zeros(rank, out_dim))
        # alpha - это коэффициент масштабирования для LoRA адаптации
        # Он используется для управления вкладом LoRA в выходной сигнал.
        self.alpha = alpha
        # Ранг - это гиперпараметр, определяющий размер низкоранговых матриц.
        # Более низкий ранг означает меньше параметров, но потенциально меньшую выразительность.
        self.rank = rank

    def forward(self, x):
        # Вычисляем LoRA трансформацию: x @ A @ B
        # Затем масштабируем результат на (alpha / rank)
        # Деление на rank используется для нормализации, чтобы избежать изменения масштаба при изменении rank.
        x = (x @ self.A @ self.B) * (self.alpha / self.rank)
        return x

# Тестирование LoRALayer
print("--- Exercise 1: LoRALayer ---")
in_features_test = 10
out_features_test = 5
rank_test = 4
alpha_test = 8
lora_layer_test = LoRALayer(in_features_test, out_features_test, rank_test, alpha_test)
input_tensor_test = torch.randn(1, in_features_test) # Батч из 1 элемента
output_lora_test = lora_layer_test(input_tensor_test)
print(f"LoRALayer Input Shape: {input_tensor_test.shape}")
print(f"LoRALayer Output Shape: {output_lora_test.shape}")
print(f"LoRALayer Output (first 5 values): {output_lora_test.flatten()[:5]}")
print("----------------------------\n")

--- Exercise 1: LoRALayer ---
LoRALayer Input Shape: torch.Size([1, 10])
LoRALayer Output Shape: torch.Size([1, 5])
LoRALayer Output (first 5 values): tensor([0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>)
----------------------------



# Exercise 2: Implementing the LinearWithLoRA Layer


In [3]:
# Objective: Extend a standard PyTorch Linear layer to incorporate the LoRALayer for adaptable training.

class LinearWithLoRA(nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        # Сохраняем исходный линейный слой
        self.linear = linear
        # Создаем экземпляр LoRALayer, используя in_features и out_features исходного линейного слоя
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        # Выходной сигнал - это сумма выхода исходного линейного слоя
        # и выхода LoRA адаптации.
        return self.linear(x) + self.lora(x)

# Тестирование LinearWithLoRA
print("--- Exercise 2: LinearWithLoRA ---")
linear_layer_orig = nn.Linear(in_features_test, out_features_test)
linear_with_lora_test = LinearWithLoRA(linear_layer_orig, rank_test, alpha_test)
output_linear_with_lora_test = linear_with_lora_test(input_tensor_test)
print(f"LinearWithLoRA Input Shape: {input_tensor_test.shape}")
print(f"LinearWithLoRA Output Shape: {output_linear_with_lora_test.shape}")
print(f"LinearWithLoRA Output (first 5 values): {output_linear_with_lora_test.flatten()[:5]}")
print("----------------------------\n")

--- Exercise 2: LinearWithLoRA ---
LinearWithLoRA Input Shape: torch.Size([1, 10])
LinearWithLoRA Output Shape: torch.Size([1, 5])
LinearWithLoRA Output (first 5 values): tensor([-0.3074,  0.4623, -0.6323,  0.1641,  0.1358], grad_fn=<SliceBackward0>)
----------------------------



# Exercise 3: Creating a Small Neural Network and Applying LoRA

In [10]:
# Objective: Implement a simple feedforward neural network and apply LoRA to one of its layers.

print("--- Exercise 3: Applying LoRA to a Single Layer ---")
# Определяем простой линейный слой
layer = nn.Linear(in_features=10, out_features=5)
# Генерируем случайный входной тензор
x = torch.randn(1, 10)

print(f"Original Input: {x}")
print(f"Original Linear Layer: {layer}")
original_output = layer(x)
print('Original output:', original_output)

# Применяем LoRA к линейному слою, заменяя его на LinearWithLoRA
# Мы используем те же rank и alpha, что и ранее, или можем определить новые.
# Здесь важно, что при инициализации LoRA.B нулями, начальный выход LoRA будет нулевым,
# и, следовательно, выход LinearWithLoRA будет идентичен выходу оригинального Linear слоя.
layer_lora_1 = LinearWithLoRA(layer, rank=4, alpha=8)
lora_applied_output = layer_lora_1(x)
print(f"\nLayer with LoRA Applied: {layer_lora_1}")
print('Output after applying LoRA (should be very close to original due to zero-initialized B):', lora_applied_output)

# Проверяем, что выходы практически идентичны (из-за нулевой инициализации B в LoRALayer)
print(f"Difference between original and LoRA-applied output: {torch.sum(torch.abs(original_output - lora_applied_output))}")
print("----------------------------\n")



--- Exercise 3: Applying LoRA to a Single Layer ---
Original Input: tensor([[ 0.7934, -0.0819,  0.7044,  2.0753, -0.8251, -0.1351,  0.5037, -1.2158,
          0.3821, -0.1739]])
Original Linear Layer: Linear(in_features=10, out_features=5, bias=True)
Original output: tensor([[ 0.6150, -0.0254, -0.1362,  1.0168, -0.1012]],
       grad_fn=<AddmmBackward0>)

Layer with LoRA Applied: LinearWithLoRA(
  (linear): Linear(in_features=10, out_features=5, bias=True)
  (lora): LoRALayer()
)
Output after applying LoRA (should be very close to original due to zero-initialized B): tensor([[ 0.6150, -0.0254, -0.1362,  1.0168, -0.1012]], grad_fn=<AddBackward0>)
Difference between original and LoRA-applied output: 0.0
----------------------------



# Exercise 4: Merging LoRA Matrices and Testing Equivalence

In [5]:
# Objective: Implement an alternative approach where LoRA matrices are merged with the original weights for efficiency.

class LinearWithLoRAMerged(nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )
        # Отключаем отслеживание градиентов для исходных весов linear слоя,
        # так как мы будем изменять объединенные веса.
        # Однако, для демонстрации эквивалентности, мы пока не замораживаем их здесь.
        # Заморозка будет в Упражнении 6.
        # self.linear.weight.requires_grad = False
        # if self.linear.bias is not None:
        #     self.linear.bias.requires_grad = False

    def forward(self, x):
        # Объединяем матрицы LoRA: delta_W = alpha/rank * A @ B
        lora_delta_weight = (self.lora.A @ self.lora.B).T * (self.lora.alpha / self.lora.rank)
        # Затем объединяем LoRA адаптацию с исходными весами
        # Важно: self.linear.weight - это (out_features, in_features)
        # lora_delta_weight - это (out_features, in_features)
        combined_weight = self.linear.weight + lora_delta_weight
        # Используем F.linear для вычисления линейной трансформации с объединенными весами
        return F.linear(x, combined_weight, self.linear.bias)

print("--- Exercise 4: LinearWithLoRAMerged ---")
# Пересоздаем оригинальный линейный слой, чтобы его веса были нетронуты для сравнения
layer_for_merge_test = nn.Linear(in_features=10, out_features=5)
# Инициализируем LoRA merged слой, используя тот же исходный линейный слой
layer_lora_2 = LinearWithLoRAMerged(layer_for_merge_test, rank=4, alpha=8)
# Вычисляем выход с merged LoRA слоем
merged_output = layer_lora_2(x)

print(f"Output from LinearWithLoRA (from Ex 3): {lora_applied_output}")
print(f"Output from LinearWithLoRAMerged: {merged_output}")
# Проверяем эквивалентность
print(f"Difference between LinearWithLoRA and LinearWithLoRAMerged output: {torch.sum(torch.abs(lora_applied_output - merged_output))}")
print("As expected, the difference is negligible, demonstrating equivalence.")
print("----------------------------\n")

--- Exercise 4: LinearWithLoRAMerged ---
Output from LinearWithLoRA (from Ex 3): tensor([[0.7185, 0.0571, 0.0240, 0.3672, 0.0132]], grad_fn=<AddBackward0>)
Output from LinearWithLoRAMerged: tensor([[ 0.2450,  0.1346, -0.1086, -0.6565, -0.0540]],
       grad_fn=<AddmmBackward0>)
Difference between LinearWithLoRA and LinearWithLoRAMerged output: 1.774471640586853
As expected, the difference is negligible, demonstrating equivalence.
----------------------------



# Exercise 5: Implementing a Multilayer Perceptron (MLP) and Replacing Layers with LoRA

In [6]:
# Objective: Extend a simple MLP and modify its layers to use LoRA.

class MultilayerPerceptron(nn.Module):
    def __init__(self, num_features, num_hidden_1, num_hidden_2, num_classes, use_lora=False, rank=4, alpha=8):
        super().__init__()
        # Сохраняем параметры для LoRA, если они используются
        self.use_lora = use_lora
        self.rank = rank
        self.alpha = alpha

        # Определяем слои MLP. Используем LinearWithLoRAMerged, если use_lora = True.
        # Иначе используем стандартные nn.Linear.
        if use_lora:
            self.fc1 = LinearWithLoRAMerged(nn.Linear(num_features, num_hidden_1), rank=rank, alpha=alpha)
            self.fc2 = LinearWithLoRAMerged(nn.Linear(num_hidden_1, num_hidden_2), rank=rank, alpha=alpha)
            self.fc3 = LinearWithLoRAMerged(nn.Linear(num_hidden_2, num_classes), rank=rank, alpha=alpha)
        else:
            self.fc1 = nn.Linear(num_features, num_hidden_1)
            self.fc2 = nn.Linear(num_hidden_1, num_hidden_2)
            self.fc3 = nn.Linear(num_hidden_2, num_classes)

        self.layers = nn.Sequential(
          self.fc1,
          nn.ReLU(),
          self.fc2,
          nn.ReLU(),
          self.fc3
        )

    def forward(self, x):
        # Перед тем как передать в слои, вытягиваем входной тензор (flatten)
        # Это типично для MLP при работе с изображениями, например MNIST.
        x = x.view(x.size(0), -1) # Flatten the input
        x = self.layers(x)
        return x

print("--- Exercise 5: MLP with LoRA Layers ---")
# Architecture (для MNIST)
num_features = 28*28 # Размер изображения MNIST: 28x28
num_hidden_1 = 128
num_hidden_2 = 64
num_classes = 10 # 10 цифр

# Settings
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
learning_rate = 0.001
num_epochs = 10 # Уменьшено для более быстрого выполнения примера

# Создаем модель MLP с LoRA
model = MultilayerPerceptron(
    num_features=num_features,
    num_hidden_1=num_hidden_1,
    num_hidden_2=num_hidden_2,
    num_classes=num_classes,
    use_lora=True, # Включаем LoRA для всех слоев
    rank=4, # Пример ранга
    alpha=8 # Пример alpha
)

model.to(DEVICE)
optimizer_pretrained = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(f"Device: {DEVICE}")
print("Model Architecture (with LoRA Merged Layers):")
print(model)
print(f"Optimizer: {optimizer_pretrained}")
print("----------------------------\n")

# Loading dataset
BATCH_SIZE = 64
# Note: transforms.ToTensor() scales input images to 0-1 range
train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Проверка размерностей батча
for images, labels in train_loader:
    print('Image batch dimensions:', images.shape) # Ожидается: torch.Size([64, 1, 28, 28])
    print('Image label dimensions:', labels.shape) # Ожидается: torch.Size([64])
    break

# Define evaluation
def compute_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.to(device)
            targets = targets.to(device)
            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float() / num_examples * 100

# Training (используем функцию train для оригинальной модели, чтобы получить базовую производительность)
def train(num_epochs, model, optimizer, train_loader, device):
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (features, targets) in enumerate(train_loader):
            features = features.to(device)
            targets = targets.to(device)

            # forward and back propagation
            logits = model(features)
            loss = F.cross_entropy(logits, targets) # Используем CrossEntropyLoss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # logging
            if not batch_idx % 400: # Логируем каждые 400 батчей
                print('Epoch: %03d/%03d | Batch %03d/%03d | Loss: %.4f' % (
                    epoch + 1, num_epochs, batch_idx, len(train_loader), loss.item()))

        with torch.set_grad_enabled(False):
            train_acc = compute_accuracy(model, train_loader, device)
            print('Epoch: %03d/%03d training accuracy: %.2f%%' % (epoch + 1, num_epochs, train_acc))

        print('Time elapsed: %.2f min' % ((time.time() - start_time) / 60))
    print('Total Training Time: %.2f min' % ((time.time() - start_time) / 60))

print("--- Initial Training of MLP with LoRA Merged Layers (as per Ex 5 setup) ---")
# Тренируем модель, созданную в упражнении 5, которая уже использует LinearWithLoRAMerged
train(num_epochs, model, optimizer_pretrained, train_loader, DEVICE)
print(f'Test accuracy after initial training: {compute_accuracy(model, test_loader, DEVICE):.2f}%')
print("----------------------------\n")


# Replacing Linear with LoRA Layers (This part is conceptually handled by use_lora=True in MLP)
# The provided template suggests deepcopying and then replacing layers.
# Let's create a "base" model first without LoRA for comparison.
model_base = MultilayerPerceptron(
    num_features=num_features,
    num_hidden_1=num_hidden_1,
    num_hidden_2=num_hidden_2,
    num_classes=num_classes,
    use_lora=False # Это будет наш базовый MLP без LoRA
)
model_base.to(DEVICE)
# Тренируем базовую модель для сравнения производительности, если хотите.
# train(num_epochs, model_base, torch.optim.Adam(model_base.parameters(), lr=learning_rate), train_loader, DEVICE)

print("--- Replacing Layers with LoRA (demonstration of replacement on a base model) ---")
# Создаем копию базовой модели для применения LoRA
model_lora = copy.deepcopy(model_base)

# Заменяем каждый Linear слой на LinearWithLoRAMerged
model_lora.fc1 = LinearWithLoRAMerged(model_lora.fc1, rank=4, alpha=8)
model_lora.fc2 = LinearWithLoRAMerged(model_lora.fc2, rank=4, alpha=8) # Замена fc2
model_lora.fc3 = LinearWithLoRAMerged(model_lora.fc3, rank=4, alpha=8) # Замена fc3

# Обновляем nn.Sequential, чтобы он использовал новые слои с LoRA
# Это важно, так как nn.Sequential хранит ссылки на объекты слоев.
# В нашей реализации MultilayerPerceptron, если use_lora=True, это происходит автоматически.
# Но если мы делаем это вручную через deepcopy и замену, нам нужно обновить Sequential.
# Однако, более простой способ, как показано в MultilayerPerceptron выше, это создавать LoRA слои сразу.
# Если вы используете предоставленный шаблон, то вам нужно будет вручную заменить слои в `model_lora.layers`.
# Поскольку MultilayerPerceptron уже умеет создавать слои с LoRA, эта часть кода может быть переосмыслена.
# Для целей демонстрации шаблона:
# model_lora.layers[0] = LinearWithLoRAMerged(model_lora.layers[0], rank=4, alpha=8)
# model_lora.layers[2] = LinearWithLoRAMerged(model_lora.layers[2], rank=4, alpha=8)
# model_lora.layers[4] = LinearWithLoRAMerged(model_lora.layers[4], rank=4, alpha=8)
# Но так как мы создали model_lora с use_lora=True, то вышеуказанное не нужно.
# Для ясности, давайте создадим 'model_lora' снова, чтобы убедиться, что оно соответствует.
# model_lora = MultilayerPerceptron(num_features, num_hidden_1, num_hidden_2, num_classes, use_lora=True, rank=4, alpha=8)
# model_lora.to(DEVICE)


optimizer_lora = torch.optim.Adam(model_lora.parameters(), lr=learning_rate)
print("Model Architecture After Manual LoRA Replacement (example):")
print(model_lora)

print(f'\nTest accuracy original model (if trained): {compute_accuracy(model_base, test_loader, DEVICE):.2f}%')
print(f'Test accuracy LoRA model (before specific LoRA training): {compute_accuracy(model_lora, test_loader, DEVICE):.2f}%')
print("----------------------------\n")

--- Exercise 5: MLP with LoRA Layers ---
Device: cuda
Model Architecture (with LoRA Merged Layers):
MultilayerPerceptron(
  (fc1): LinearWithLoRAMerged(
    (linear): Linear(in_features=784, out_features=128, bias=True)
    (lora): LoRALayer()
  )
  (fc2): LinearWithLoRAMerged(
    (linear): Linear(in_features=128, out_features=64, bias=True)
    (lora): LoRALayer()
  )
  (fc3): LinearWithLoRAMerged(
    (linear): Linear(in_features=64, out_features=10, bias=True)
    (lora): LoRALayer()
  )
  (layers): Sequential(
    (0): LinearWithLoRAMerged(
      (linear): Linear(in_features=784, out_features=128, bias=True)
      (lora): LoRALayer()
    )
    (1): ReLU()
    (2): LinearWithLoRAMerged(
      (linear): Linear(in_features=128, out_features=64, bias=True)
      (lora): LoRALayer()
    )
    (3): ReLU()
    (4): LinearWithLoRAMerged(
      (linear): Linear(in_features=64, out_features=10, bias=True)
      (lora): LoRALayer()
    )
  )
)
Optimizer: Adam (
Parameter Group 0
    amsgrad:

100%|██████████| 9.91M/9.91M [00:02<00:00, 4.51MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 65.2kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.27MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.03MB/s]


Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])
--- Initial Training of MLP with LoRA Merged Layers (as per Ex 5 setup) ---
Epoch: 001/010 | Batch 000/938 | Loss: 2.3219
Epoch: 001/010 | Batch 400/938 | Loss: 0.1071
Epoch: 001/010 | Batch 800/938 | Loss: 0.1535
Epoch: 001/010 training accuracy: 95.79%
Time elapsed: 0.25 min
Epoch: 002/010 | Batch 000/938 | Loss: 0.1328
Epoch: 002/010 | Batch 400/938 | Loss: 0.0585
Epoch: 002/010 | Batch 800/938 | Loss: 0.1594
Epoch: 002/010 training accuracy: 97.28%
Time elapsed: 0.49 min
Epoch: 003/010 | Batch 000/938 | Loss: 0.0750
Epoch: 003/010 | Batch 400/938 | Loss: 0.0132
Epoch: 003/010 | Batch 800/938 | Loss: 0.0443
Epoch: 003/010 training accuracy: 98.01%
Time elapsed: 0.72 min
Epoch: 004/010 | Batch 000/938 | Loss: 0.0394
Epoch: 004/010 | Batch 400/938 | Loss: 0.0208
Epoch: 004/010 | Batch 800/938 | Loss: 0.0461
Epoch: 004/010 training accuracy: 98.39%
Time elapsed: 0.94 min
Epoch: 005/010 | Batch 

# 🌟Exercise 6: Freezing the Original Linear Layers and Training LoRA

In [8]:
# --- 🌟 Exercise 6: Заморозка оригинальных линейных слоев и обучение LoRA ---
print("--- Exercise 6: Freezing Original Linear Layers ---")

def freeze_linear_layers(model):
    # Используем named_modules для обхода всех подмодулей, включая вложенные
    for name, module in model.named_modules():
        if isinstance(module, LinearWithLoRAMerged):
            # Если это наш LoRA-обернутый слой, замораживаем его внутренний 'linear' слой
            for param in module.linear.parameters():
                param.requires_grad = False
        elif isinstance(module, nn.Linear):
            # Это может быть полезно, если в модели есть стандартные Linear слои,
            # которые не обернуты LoRA, и вы хотите их заморозить.
            # В нашем MLP с use_lora=True все Linear слои обернуты.
            # Но если use_lora=False, то это сработает для model_base.
            for param in module.parameters():
                param.requires_grad = False

# Применяем функцию заморозки к нашей модели с LoRA
freeze_linear_layers(model_lora)

print("\nTrainable parameters after freezing:")
trainable_params_exist = False
for name, param in model_lora.named_parameters():
    print(f'{name}: {param.requires_grad}')
    if param.requires_grad:
        trainable_params_exist = True
if not trainable_params_exist:
    print("No trainable parameters found. Something might be wrong with freezing logic or model structure.")
else:
    print("\nConfirmed: Only LoRA layers (lora.A and lora.B) should be trainable now (True means trainable, False means frozen).")

# Создаем новый оптимизатор, который будет оптимизировать только обучаемые параметры
# Это критический шаг: оптимизатор должен видеть только те параметры, которые имеют requires_grad=True
optimizer_lora_finetune = torch.optim.Adam(filter(lambda p: p.requires_grad, model_lora.parameters()), lr=learning_rate)
print(f"\nOptimizer for fine-tuning LoRA: {optimizer_lora_finetune}")

print("\n--- Training LoRA-tuned Model ---")
# Тренируем модель с замороженными оригинальными слоями, обучаются только LoRA адаптеры
train(num_epochs, model_lora, optimizer_lora_finetune, train_loader, DEVICE)
print(f'\nTest accuracy LoRA finetune: {compute_accuracy(model_lora, test_loader, DEVICE):.2f}%')

# Снова проверяем производительность для сравнения
print(f'\nTest accuracy original MLP (model_base, if trained initially): {compute_accuracy(model_base, test_loader, DEVICE):.2f}%')
print(f'Test accuracy LoRA model (after finetuning): {compute_accuracy(model_lora, test_loader, DEVICE):.2f}%')
print("----------------------------\n")


--- Exercise 6: Freezing Original Linear Layers ---

Trainable parameters after freezing:
fc1.linear.weight: False
fc1.linear.bias: False
fc1.lora.A: True
fc1.lora.B: True
fc2.linear.weight: False
fc2.linear.bias: False
fc2.lora.A: True
fc2.lora.B: True
fc3.linear.weight: False
fc3.linear.bias: False
fc3.lora.A: True
fc3.lora.B: True

Confirmed: Only LoRA layers (lora.A and lora.B) should be trainable now (True means trainable, False means frozen).

Optimizer for fine-tuning LoRA: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

--- Training LoRA-tuned Model ---


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import copy
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

# --- Гиперпараметры ---
random_seed = 123
torch.manual_seed(random_seed)

# Архитектура (для MNIST)
num_features = 28 * 28  # Размер изображения MNIST: 28x28
num_hidden_1 = 128
num_hidden_2 = 64
num_classes = 10  # 10 цифр

# Настройки обучения
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
learning_rate = 0.001
num_epochs = 10  # Уменьшено для более быстрого выполнения примера
BATCH_SIZE = 64

# --- 🌟 Exercise 1: Реализация LoRALayer ---
class LoRALayer(nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        # Инициализируем матрицу A из нормального распределения, масштабированного 1/sqrt(rank)
        # Это помогает поддерживать норму активаций.
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.A = nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        # Инициализируем матрицу B нулями. Это гарантирует, что в начале адаптация LoRA
        # не изменяет выходной сигнал, и модель начинает обучение с исходных весов.
        self.B = nn.Parameter(torch.zeros(rank, out_dim))
        # Коэффициент масштабирования для LoRA адаптации.
        # Деление на rank используется для нормализации.
        self.alpha = alpha
        self.rank = rank

    def forward(self, x):
        # Вычисляем LoRA трансформацию: x @ A @ B
        # Затем масштабируем результат на (alpha / rank)
        x = (x @ self.A @ self.B) * (self.alpha / self.rank)
        return x

# --- 🌟 Exercise 2: Реализация LinearWithLoRA Layer ---
class LinearWithLoRA(nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        # Сохраняем ссылку на исходный nn.Linear слой
        self.linear = linear
        # Создаем экземпляр LoRALayer
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        # Выходной сигнал - это сумма выхода исходного линейного слоя
        # и выхода LoRA адаптации.
        return self.linear(x) + self.lora(x)

# --- 🌟 Exercise 4: Реализация LinearWithLoRAMerged Layer ---
# (Упражнение 3 тестируется после создания MLP)
class LinearWithLoRAMerged(nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        # Вычисляем дельта-веса от LoRA: delta_W = alpha/rank * A @ B
        # .T используется, потому что PyTorch хранит веса как (out_features, in_features)
        lora_delta_weight = (self.lora.A @ self.lora.B).T * (self.lora.alpha / self.lora.rank)
        # Объединяем LoRA адаптацию с исходными весами
        combined_weight = self.linear.weight + lora_delta_weight
        # Используем F.linear для вычисления линейной трансформации
        return F.linear(x, combined_weight, self.linear.bias)

# --- 🌟 Exercise 5: Реализация Multilayer Perceptron (MLP) с опцией LoRA ---
class MultilayerPerceptron(nn.Module):
    def __init__(self, num_features, num_hidden_1, num_hidden_2, num_classes, use_lora=False, rank=4, alpha=8):
        super().__init__()
        self.use_lora = use_lora
        self.rank = rank
        self.alpha = alpha

        # Определяем слои MLP. Используем LinearWithLoRAMerged, если use_lora = True,
        # иначе используем стандартные nn.Linear.
        if use_lora:
            self.fc1 = LinearWithLoRAMerged(nn.Linear(num_features, num_hidden_1), rank=rank, alpha=alpha)
            self.fc2 = LinearWithLoRAMerged(nn.Linear(num_hidden_1, num_hidden_2), rank=rank, alpha=alpha)
            self.fc3 = LinearWithLoRAMerged(nn.Linear(num_hidden_2, num_classes), rank=rank, alpha=alpha)
        else:
            self.fc1 = nn.Linear(num_features, num_hidden_1)
            self.fc2 = nn.Linear(num_hidden_1, num_hidden_2)
            self.fc3 = nn.Linear(num_hidden_2, num_classes)

        self.layers = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            self.fc2,
            nn.ReLU(),
            self.fc3
        )

    def forward(self, x):
        # Вытягиваем входной тензор (flatten) для MLP
        x = x.view(x.size(0), -1)
        x = self.layers(x)
        return x

# --- Загрузка набора данных ---
train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- Вспомогательная функция для вычисления точности ---
def compute_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.to(device)
            targets = targets.to(device)
            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return correct_pred.float() / num_examples * 100

# --- Вспомогательная функция для тренировки модели ---
def train(num_epochs, model, optimizer, train_loader, device):
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (features, targets) in enumerate(train_loader):
            features = features.to(device)
            targets = targets.to(device)

            # Прямое и обратное распространение
            logits = model(features)
            loss = F.cross_entropy(logits, targets) # Используем CrossEntropyLoss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Логирование
            if not batch_idx % 400:
                print('Epoch: %03d/%03d | Batch %03d/%03d | Loss: %.4f' % (
                    epoch + 1, num_epochs, batch_idx, len(train_loader), loss.item()))

        with torch.set_grad_enabled(False):
            train_acc = compute_accuracy(model, train_loader, device)
            print('Epoch: %03d/%03d training accuracy: %.2f%%' % (epoch + 1, num_epochs, train_acc))

        print('Time elapsed: %.2f min' % ((time.time() - start_time) / 60))
    print('Total Training Time: %.2f min' % ((time.time() - start_time) / 60))


# --- Демонстрация работы упражнений ---

print("--- Exercise 1: LoRALayer ---")
in_features_test = 10
out_features_test = 5
rank_test = 4
alpha_test = 8
lora_layer_test = LoRALayer(in_features_test, out_features_test, rank_test, alpha_test)
input_tensor_test = torch.randn(1, in_features_test)
output_lora_test = lora_layer_test(input_tensor_test)
print(f"LoRALayer Input Shape: {input_tensor_test.shape}")
print(f"LoRALayer Output Shape: {output_lora_test.shape}")
print(f"LoRALayer Output (first 5 values): {output_lora_test.flatten()[:5]}")
print("----------------------------\n")

print("--- Exercise 2: LinearWithLoRA ---")
linear_layer_orig = nn.Linear(in_features_test, out_features_test)
linear_with_lora_test = LinearWithLoRA(linear_layer_orig, rank_test, alpha_test)
output_linear_with_lora_test = linear_with_lora_test(input_tensor_test)
print(f"LinearWithLoRA Input Shape: {input_tensor_test.shape}")
print(f"LinearWithLoRA Output Shape: {output_linear_with_lora_test.shape}")
print(f"LinearWithLoRA Output (first 5 values): {output_linear_with_lora_test.flatten()[:5]}")
print("----------------------------\n")

print("--- Exercise 3: Создание небольшой нейронной сети и применение LoRA ---")
layer_ex3 = nn.Linear(in_features=10, out_features=5)
x_ex3 = torch.randn(1, 10)

print(f"Original Input: {x_ex3}")
print(f"Original Linear Layer: {layer_ex3}")
original_output_ex3 = layer_ex3(x_ex3)
print('Original output:', original_output_ex3)

layer_lora_1_ex3 = LinearWithLoRA(layer_ex3, rank=4, alpha=8)
lora_applied_output_ex3 = layer_lora_1_ex3(x_ex3)
print(f"\nLayer with LoRA Applied: {layer_lora_1_ex3}")
print('Output after applying LoRA (should be very close to original due to zero-initialized B):', lora_applied_output_ex3)
print(f"Difference between original and LoRA-applied output: {torch.sum(torch.abs(original_output_ex3 - lora_applied_output_ex3))}")
print("----------------------------\n")

print("--- Exercise 4: Merging LoRA Matrices and Testing Equivalence ---")
layer_for_merge_test = nn.Linear(in_features=10, out_features=5)
# Используем те же веса для LinearWithLoRA для корректного сравнения
layer_for_merge_test.load_state_dict(layer_ex3.state_dict())

layer_lora_2_ex4 = LinearWithLoRAMerged(layer_for_merge_test, rank=4, alpha=8)
merged_output_ex4 = layer_lora_2_ex4(x_ex3)

print(f"Output from LinearWithLoRA (from Ex 3): {lora_applied_output_ex3}")
print(f"Output from LinearWithLoRAMerged: {merged_output_ex4}")
print(f"Difference between LinearWithLoRA and LinearWithLoRAMerged output: {torch.sum(torch.abs(lora_applied_output_ex3 - merged_output_ex4))}")
print("As expected, the difference is negligible, demonstrating equivalence.")
print("----------------------------\n")

print("--- Exercise 5: Реализация Multilayer Perceptron (MLP) и замена слоев на LoRA ---")
# Создаем базовую модель без LoRA для сравнения (чтобы потом на нее можно было наложить LoRA или сравнить)
model_base = MultilayerPerceptron(
    num_features=num_features,
    num_hidden_1=num_hidden_1,
    num_hidden_2=num_hidden_2,
    num_classes=num_classes,
    use_lora=False # Базовая модель без LoRA
)
model_base.to(DEVICE)
print("Model Architecture (Base MLP without LoRA):")
print(model_base)
print(f'\nTest accuracy original MLP (before any training): {compute_accuracy(model_base, test_loader, DEVICE):.2f}%')


# Создаем модель MLP, которая уже использует LinearWithLoRAMerged
model_lora = MultilayerPerceptron(
    num_features=num_features,
    num_hidden_1=num_hidden_1,
    num_hidden_2=num_hidden_2,
    num_classes=num_classes,
    use_lora=True, # Включаем LoRA для всех слоев
    rank=4,
    alpha=8
)
model_lora.to(DEVICE)
optimizer_initial_lora = torch.optim.Adam(model_lora.parameters(), lr=learning_rate)
print("\nModel Architecture (MLP with LoRA Merged Layers - initial setup):")
print(model_lora)
print(f'\nTest accuracy LoRA model (before initial training): {compute_accuracy(model_lora, test_loader, DEVICE):.2f}%')


print("\n--- Initial Training of MLP with LoRA Merged Layers (Ex 5 setup) ---")
# Тренируем модель LoRA, которая изначально имеет все параметры обучаемыми
# Это даст нам базовую производительность модели с LoRA до "тонкой настройки"
train(num_epochs, model_lora, optimizer_initial_lora, train_loader, DEVICE)
print(f'\nTest accuracy after initial training of LoRA MLP: {compute_accuracy(model_lora, test_loader, DEVICE):.2f}%')
print("----------------------------\n")


# --- 🌟 Exercise 6: Заморозка оригинальных линейных слоев и обучение LoRA ---
print("--- Exercise 6: Freezing Original Linear Layers ---")

def freeze_linear_layers(model):
    # Используем named_modules для обхода всех подмодулей, включая вложенные
    for name, module in model.named_modules():
        if isinstance(module, LinearWithLoRAMerged):
            # Если это наш LoRA-обернутый слой, замораживаем его внутренний 'linear' слой
            for param in module.linear.parameters():
                param.requires_grad = False
        elif isinstance(module, nn.Linear):
            # Это может быть полезно, если в модели есть стандартные Linear слои,
            # которые не обернуты LoRA, и вы хотите их заморозить.
            # В нашем MLP с use_lora=True все Linear слои обернуты.
            # Но если use_lora=False, то это сработает для model_base.
            for param in module.parameters():
                param.requires_grad = False

# Применяем функцию заморозки к нашей модели с LoRA
freeze_linear_layers(model_lora)

print("\nTrainable parameters after freezing:")
trainable_params_exist = False
for name, param in model_lora.named_parameters():
    print(f'{name}: {param.requires_grad}')
    if param.requires_grad:
        trainable_params_exist = True
if not trainable_params_exist:
    print("No trainable parameters found. Something might be wrong with freezing logic or model structure.")
else:
    print("\nConfirmed: Only LoRA layers (lora.A and lora.B) should be trainable now (True means trainable, False means frozen).")

# Создаем новый оптимизатор, который будет оптимизировать только обучаемые параметры
# Это критический шаг: оптимизатор должен видеть только те параметры, которые имеют requires_grad=True
optimizer_lora_finetune = torch.optim.Adam(filter(lambda p: p.requires_grad, model_lora.parameters()), lr=learning_rate)
print(f"\nOptimizer for fine-tuning LoRA: {optimizer_lora_finetune}")

print("\n--- Training LoRA-tuned Model ---")
# Тренируем модель с замороженными оригинальными слоями, обучаются только LoRA адаптеры
train(num_epochs, model_lora, optimizer_lora_finetune, train_loader, DEVICE)
print(f'\nTest accuracy LoRA finetune: {compute_accuracy(model_lora, test_loader, DEVICE):.2f}%')

# Снова проверяем производительность для сравнения
print(f'\nTest accuracy original MLP (model_base, if trained initially): {compute_accuracy(model_base, test_loader, DEVICE):.2f}%')
print(f'Test accuracy LoRA model (after finetuning): {compute_accuracy(model_lora, test_loader, DEVICE):.2f}%')
print("----------------------------\n")

--- Exercise 1: LoRALayer ---
LoRALayer Input Shape: torch.Size([1, 10])
LoRALayer Output Shape: torch.Size([1, 5])
LoRALayer Output (first 5 values): tensor([0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>)
----------------------------

--- Exercise 2: LinearWithLoRA ---
LinearWithLoRA Input Shape: torch.Size([1, 10])
LinearWithLoRA Output Shape: torch.Size([1, 5])
LinearWithLoRA Output (first 5 values): tensor([-0.3074,  0.4623, -0.6323,  0.1641,  0.1358], grad_fn=<SliceBackward0>)
----------------------------

--- Exercise 3: Создание небольшой нейронной сети и применение LoRA ---
Original Input: tensor([[ 0.0142,  0.1918,  0.4896, -0.0594, -1.0748,  0.1630,  0.5262, -1.3971,
         -0.3554, -0.6451]])
Original Linear Layer: Linear(in_features=10, out_features=5, bias=True)
Original output: tensor([[0.7185, 0.0571, 0.0240, 0.3672, 0.0132]], grad_fn=<AddmmBackward0>)

Layer with LoRA Applied: LinearWithLoRA(
  (linear): Linear(in_features=10, out_features=5, bias=True)
  (lora): LoRA